<a href="https://colab.research.google.com/github/k-metrics/DAWS2020/blob/master/tidyr%E3%81%A8%E9%9A%A0%E3%81%95%E3%82%8C%E3%81%9F%E4%BE%BF%E5%88%A9%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tidyrと隠された便利関数
---



In [ ]:
library(tidyverse)

## 欠損と欠落
分析対象とするデータの一部が失われていることはよくあり、逆に綺麗にそろっている方が珍しいと言えます。
データが失われているというのは

* 観測値の一部がない（セルに抜けがある）
* 観測記録の一部がない（行に抜けがある）

という状態であり、ここでは前者を「欠損」、後者を「欠落」と便宜的に呼びます。他で、この呼び方で通じるかは不明（たぶん、通じない）。  

　  
このセッションでは、このような欠損や欠落を処理する場合に非常に便利だけどあまり紹介されない`tidyr`パッケージの関数を取り上げます。

### 欠損の例
**欠損**が含まれているデータセットとして有名なのが、ニューヨークの大気観測データをまとめたデータセット`airquality`です。1973年の5月1日から9月30日までの連続した153日間の観測データが収められており、日付の抜けはありません。  
実行するとデータが表示が長くなるので最初と最後の数行のみを表示。

In [ ]:
airquality %>% head()
airquality %>% tail()

### 欠落の例
欠落が含まれているデータ（セット）としては新型コロナウィルス検査結果のデータがあげられます。陽性確定者がゼロの日はレコード（インスタンス）が追加されないので、集計してもゼロの日はレコード（インスタンス）がありません。  

In [ ]:
df <- "https://gist.githubusercontent.com/k-metrics/c963351ca993f84acde7bba38139f046/raw/8d390198a6d9bd9a4c3e5050d86856638d872949/covid19.csv" %>% 
  readr::read_csv() %>%
  dplyr::mutate(n = as.integer(n))
df

このようなデータ（セット）は読み手が、「ゼロだから記録として出てこない」と読み取ってくれるだろうという暗黙の了解の元で作成されています。  
なので、ここでは、このような欠落を「暗黙的な欠落」と呼びます。他で…（略）。

### 暗黙的な欠落の問題点
では、このような暗黙的な欠落はデータ分析において、どのような問題を引き起こすのか見てみます。  
上記のデータを用いて折線グラフを描いてみましょう。

In [ ]:
df %>% 
  ggplot2::ggplot(ggplot2::aes(x = date, y = n)) + 
    ggplot2::geom_line()

### 【質問】
描いた折線グラフにどのような問題があるか考えてください。

■挙手または指名で回答■

### 【質問】
新型コロナウィルスに関するデータセットが持っている暗示的な欠落を明示的な欠落に変換する手順を考えて、その手順を説明してください（Rの関数名などを出す必要はありません）。  
「明示的な欠落」とは、データが欠落していることが分かるようにすることを意味しています。  
　  
※ 言い換えると「正しいグラフが描けるようにするには、どうしたらいいか」です。

### 【回答】
■挙手または指名で回答■

### 【処理例】
実際に以下のコードを実行して、どのように変換されるかを確認してください。

In [ ]:
cal_data <- data.frame(date = seq.Date(from = lubridate::as_date("2020/1/15"),
                                       to = lubridate::as_date("2020/2/4"),
                                       by = "day"))
cal_data %>% head()

dplyr::left_join(cal_data, df, by = c("date")) %>% 
      dplyr::mutate(n = dplyr::if_else(is.na(n), 0L, n))

## 欠落の変換
前述の処理を簡単にしてくれるのが`tidyr::complete`関数です。とりあえず実行してみましょう。

In [ ]:
df %>% 
  tidyr::complete(date = seq.Date(min(date), max(date), by = "day"),
                  fill = list(n = 0L))

明示的な欠落に変換したデータでグラフを描いて、最初のグラフと比較してみてください。

In [ ]:
df %>% 
  tidyr::complete(date = seq.Date(min(date), max(date), by = "day"),
                  fill = list(n = 0L)) %>% 
  ggplot2::ggplot(ggplot2::aes(x = date, y = n)) + 
    ggplot2::geom_line()

### 組み合わせデータの変換
先程の例は比較的単純な変換処理でしたが、複数の変量の組み合わせを変換（補完）するにはどのようにすれば良いでしょうか？  
以下のデータで考えてみましょう。

In [ ]:
x <- data.frame(group = c(1:2, 1), item_id = c(1:2, 2),
                item_name = c("a", "b", "b"),
                value1 = 1:3, value2 = 4:6)
x

### 【質問】
本来であれば`group`と`item_id`の全ての組み合わせ（4通りの組み合わせ）があるべきデータです。このデータの暗示的な欠落を明示的な欠落に変換する手順を考えて、その手順を説明してください（Rの関数名などを出す必要はありません）。


### 【回答】
■挙手または指名で回答■

### 【処理例】
実際に以下のコードを実行して、どのように変換されるかを確認してください。

In [ ]:
x %>%
   with(expand.grid(group = unique(group), item_id = unique(item_id))) %>% 
   dplyr::left_join(x)

### 【Tips】

データを良く見ると`item_id`と`item_name`は連動しているようですので、合わせて変換（補完）したいです。このような場合に`tidyr::nesting`関数を用いてパラメータを指定します。

In [ ]:
x %>% 
  tidyr::complete(group, tidyr::nesting(item_id, item_name))

## 欠損値の変換
新型コロナウィルスデータのように欠損値（`NA`）が0（ゼロ）と等価であるような場合には、欠損値を一括して変換したいものです。  
　  
このような場合は`tidyr::replace_na`関数を使うと一括変換が可能です。

In [ ]:
data.frame(x = c(1, 2, NA), y = c("a", NA, "b")) %>% 
   print() %>% 
   tidyr::replace_na(list(x = 0, y = "unknown"))

### 規則的な欠損を変換する
Excelで作成されたようなデータではセル結合の弊害から規則的に欠損していることがあります。これも一種の暗黙的な了解の元で欠損させているのですが、処理的に困ることがあります。  
例えば、総務省が公開している「都道府県コード及び市町村コード」改正一覧表は規則的な欠損が存在しています。

In [ ]:
# パスから直接読み込むことができない場合は、一旦、ダウンロードしてから読み込む
path <- "https://www.soumu.go.jp/main_content/000562731.xls"
file <- paste0("./", basename(path))
path %>%
  download.file(file)

# 今回は列を最小限に絞っておく
city_code <- file %>% 
  readxl::read_excel(col_names = FALSE, skip = 4) %>%
  dplyr::select(5:7, 11:12, 14) %>%
  dplyr::rename(`都道府県名` = ...5, `旧コード` = ...6, `旧市町村名` = ...7, `コード` = ...11, `市町村名` = ...12, `改正事由` = ...14)

city_code %>% head(10)

都道府県の列は同上の場合、都道府県名が省略されています。データ処理のためには都道府県名の欠損を変換して、正しい都道府県名を設定する必要があります。  
このような場合に便利なのが`tdiyr::fill`関数です。

In [ ]:
city_code %>%
  tidyr::fill(`都道府県名`) %>% 
  head(10)

ついでに「同左」と「改正事由」の欠損値を変換しておきます。

In [ ]:
city_code %>%
  tidyr::fill(`都道府県名`) %>% 
  dplyr::mutate(`コード` = dplyr::if_else(`コード` == "同左", `旧コード`, `コード`),
                `市町村名` = dplyr::if_else(`市町村名` == "同左", `旧市町村名`, `市町村名`)) %>%
  dplyr::filter(`市町村名` != "削除") %>%
  tidyr::replace_na(list(`改正事由` = "新設"))

## 欠損値の削除
`NA`を含む行を削除する場合には`tdiyr::drop_na`関数を使います。パラメータの指定方法で結果が変わりますので注意してください。

In [ ]:
airquality %>% head()

In [ ]:
airquality %>% 
   tidyr::drop_na() %>%
   head()

In [ ]:
airquality %>% 
   tidyr::drop_na(Ozone) %>%
   head()

# その他の便利な関数
その他の便利な関数についてはチュートリアルファイルを用意してあります。入手方法や具体的な使い方などは、来月のLT大会の時に紹介します。

本資料のコードの一部は各関数のヘルプに記載されているサンプルコードを用いています。  

---

Enjoy!

CC 4.0 BY-NC-SA, Sampo Suzuki